# label as order of finish
the label should be resricted to a 0 - 1 range taking into account the number of horses per race
so a horse that places 5th in a race of 10 horses gets a score of 0.5

$
\text{label} = 1 - \dfrac{(\text{finish order} - 1)}{(\text{number of horses in race} - 1)}
$

In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from IPython.display import display

sys.path.insert(0, '..')
np.random.seed(42)
pd.options.display.max_columns = None
%matplotlib inline

In [2]:
from scripts.utils import read_netkeiba

df = read_netkeiba()

In [3]:
df['r_contender_count'] = df.groupby('r_id').c_id.count().loc[df.r_id].values
df['c_norm_order_of_finish'] = 1.0 - (df.c_order_of_finish - 1) / (df.r_contender_count - 1)

idx_cols = ['c_id', 'r_id', 'h_id', 'j_id', 't_id', 'r_key', 'r_url', 'h_key', 'h_url', 'j_key', 
            'j_url', 't_key', 't_url', 'r_date']

X = df.drop(columns=idx_cols + ['c_norm_order_of_finish', 'c_order_of_finish', 'c_finish_time'])
y = df.c_norm_order_of_finish

X.shape, y.shape

((240471, 51), (240471,))

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in sss.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((192376, 51), (192376,), (48095, 51), (48095,))

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # df.h_birthday = pd.to_datetime(df.h_birthday)
        # df.r_date = pd.to_datetime(df.r_date)
        # df['h_age_days'] = df.r_date - df.h_birthday
        return []

# from sklearn.pipeline import Pipeline